In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import wandb
import sys
import os
from datasets import Dataset
from typing import List, Sequence, Iterator, Generator, Tuple
import evaluate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
import numpy as np

2025-03-12 09:23:21.265690: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-12 09:23:21.405355: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-12 09:23:22.393282: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
from datasets import Dataset

In [3]:
sys.path.append(os.path.join(os.path.abspath(os.getcwd()), ".."))

In [4]:
from certainty import EventSentence, EventType, load_events, id2label, label2id


In [5]:
train_events, dev_events, test_events = load_events('en_train.json', 'en_dev.json', 'en_test.json')

In [11]:
import pandas as pd

In [19]:
Dataset.from_pandas(pd.DataFrame(train_events)[["label", "text", "trigger", "type", "genericity", "polarity"]])

Dataset({
    features: ['label', 'text', 'trigger', 'type', 'genericity', 'polarity'],
    num_rows: 4261
})

In [7]:
trainset = Dataset.from_list(train_events)

ArrowInvalid: cannot mix list and non-list, non-null values

In [6]:
train_events[0]

{'sent_id': 'bc/CNN_IP_20030329.1600.02/001',
 'text': 'It was in northern Iraq today that an eight artillery round hit the site occupied by Kurdish fighters near Chamchamal',
 'type': 'Attack',
 'modality': 'Asserted',
 'label': 1,
 'polarity': 'Positive',
 'genericity': 'Specific',
 'trigger': 'hit'}

In [7]:
trainset: Dataset = Dataset.from_list(train_events)

In [8]:
devset: Dataset = Dataset.from_list(dev_events)

In [9]:
testset: Dataset = Dataset.from_list(test_events)

In [10]:
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased', cache_dir="../cache")

In [11]:
def preprocess_function(data: Dataset):
    return tokenizer(data["features"], truncation=True)

In [12]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

In [14]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
            "precision":precision.compute(predictions=predictions, references=labels)["precision"], 
            "recall":recall.compute(predictions=predictions, references=labels)["recall"], 
            "f1":f1.compute(predictions=predictions, references=labels)["f1"], 
           }

In [15]:
def concat_trigger(data: Dataset):
    data["features"] = data["trigger"] + " " + data["text"]
    return data
    
tokenized_train: Dataset = trainset.map(concat_trigger, batched=False).map(preprocess_function, batched=True)
tokenized_dev: Dataset = devset.map(concat_trigger, batched=False).map(preprocess_function, batched=True)
tokenized_test: Dataset = testset.map(concat_trigger, batched=False).map(preprocess_function, batched=True)


model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", cache_dir="../cache", num_labels=2, id2label=id2label, label2id=label2id
)

wandb.init(
    project="certainty",
    name="bert-baseline-using-trigger",
    dir="..",
)

training_args = TrainingArguments(
    output_dir="../models/bert_baseline",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
wandb.finish()

Map:   0%|          | 0/4261 [00:00<?, ? examples/s]

Map:   0%|          | 0/4261 [00:00<?, ? examples/s]

Map:   0%|          | 0/496 [00:00<?, ? examples/s]

Map:   0%|          | 0/496 [00:00<?, ? examples/s]

Map:   0%|          | 0/424 [00:00<?, ? examples/s]

Map:   0%|          | 0/424 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: Currently logged in as: peder-august. Use `wandb login --relogin` to force relogin


/home/peder/.pyenv/versions/3.10.13/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.444957,0.792339,0.813896,0.921348,0.864295
2,0.384900,0.434821,0.816532,0.865014,0.882022,0.873435


eval/accuracy,▁█
eval/f1,▁█
eval/loss,█▁
eval/precision,▁█
eval/recall,█▁
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,█▁
train/epoch,▁▇██
train/global_step,▁▇██
train/grad_norm,▁


In [18]:

def concat_event_type(data: Dataset):
    data["features"] = data["type"] + " " + data["text"]
    return data
    
tokenized_train: Dataset = trainset.map(concat_event_type, batched=False).map(preprocess_function, batched=True)
tokenized_dev: Dataset = devset.map(concat_event_type, batched=False).map(preprocess_function, batched=True)
tokenized_test: Dataset = testset.map(concat_event_type, batched=False).map(preprocess_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", cache_dir="../cache", num_labels=2, id2label=id2label, label2id=label2id
)

wandb.init(
    project="certainty",
    name="bert-baseline-using-event-type",
    dir="..",
)

training_args = TrainingArguments(
    output_dir="../models/bert_baseline",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
wandb.finish()

Map:   0%|          | 0/4261 [00:00<?, ? examples/s]

Map:   0%|          | 0/4261 [00:00<?, ? examples/s]

Map:   0%|          | 0/496 [00:00<?, ? examples/s]

Map:   0%|          | 0/496 [00:00<?, ? examples/s]

Map:   0%|          | 0/424 [00:00<?, ? examples/s]

Map:   0%|          | 0/424 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: Currently logged in as: peder-august. Use `wandb login --relogin` to force relogin


/home/peder/.pyenv/versions/3.10.13/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.475227,0.774194,0.793269,0.926966,0.854922
2,0.400200,0.469655,0.790323,0.836898,0.879213,0.857534


eval/accuracy,▁█
eval/f1,▁█
eval/loss,█▁
eval/precision,▁█
eval/recall,█▁
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,█▁
train/epoch,▁▇██
train/global_step,▁▇██
train/grad_norm,▁
